In [ ]:
import torch.optim as optim
import torchlensmaker as tlm

In [ ]:

lens_width = 15.0
surface1 = tlm.Parabola(lens_width, init=[-0.005])
surface2 = tlm.Parabola.share(surface1, scale=-1.0)

optics = tlm.OpticalStack([
    tlm.ParallelBeamRandom(radius=0.9*lens_width),
    tlm.FixedGap(15.),
    
    tlm.RefractiveSurface(surface1, (1.0, 1.49)),
    tlm.FixedGap(5.),
    tlm.RefractiveSurface(surface2, (1.49, 1.0)),
    
    tlm.FixedGap(80.0),
    tlm.FocalPointLoss(),
])


tlm.render(optics, num_rays=10)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    num_rays = 20,
    num_iter = 150
)

tlm.render(optics, num_rays=10)



In [ ]:
lens = tlm.Lens(optics.stack[2], optics.stack[3], optics.stack[4])

part = tlm.lens_to_part(lens)
part